# 20220518

开始第三版带有晶界模型的测试

首先生成一个210晶界模型


In [ ]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix

li3clo_210=Structure.from_file("CONTCAR")
li3clo_210.to("cif","210.cif")
li3clo_210.add_oxidation_state_by_guess()

#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})
#local_env_finder = CutOffDictNN({("Li+","Cl-"):2.8})
np.set_printoptions(precision=2,suppress=True)  
site_to_mark1=[]
site_to_mark2=[]
site_to_mark3=[]
for i in range(0,len(li3clo_210)):
    if "Li+" not in li3clo_210[i].species:
        continue
    
    #print(local_env_finder.get_nn_info(li3clo_210,i))
    print(len(local_env_finder.get_nn_info(li3clo_210,i)))
    """
    
    for #local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
        if len(local_env_finder.get_nn_info(li3clo_210,i))==5:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"Na+":1.0})

    if len(local_env_finder.get_nn_info(li3clo_210,i))==4:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"K+":1.0})
    """
    
    if len(local_env_finder.get_nn_info(li3clo_210,i))!=12:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        print(build_distance_matrix_from_getnninfo_output(local_env_finder.get_nn_info(li3clo_210,i)))
        site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
        li_num=0
        for site in site_contain_li:
            if "Li+" in site:
                li_num+=1
        print("Li num",li_num)

        if len(local_env_finder.get_nn_info(li3clo_210,i))==11:
            site_to_mark1.append(i)
        elif len(local_env_finder.get_nn_info(li3clo_210,i))==8:
            site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
            li_num=0
            for site in site_contain_li:
                if "Li+" in site:
                    li_num+=1
            if li_num==6:
                site_to_mark2.append(i)
            else:
                site_to_mark3.append(i)
        else:
            raise ValueError()

for i in site_to_mark1:
    li3clo_210.replace(i,{"Na+":1.0})
for i in site_to_mark2:
    li3clo_210.replace(i,{"K+":1.0})
for i in site_to_mark3:
    li3clo_210.replace(i,{"Rb+":1.0})
li3clo_210.to("cif","marked210.cif")
    
    #print(local_env_finder.get_all_nn_info(li3clo_210))



For 210 grain boundary, there are several types of Li environment

1. General Lithium (larger green atom): 8 Li+ and 4 Cl- and 2 O2-
2. Gold: 7 Li+ and 4 Cl- and 2O2-
3. Pink: 4 Li+ and 4 Cl- and 1O2-
4. Purple: 6 Li+ and 2 Cl-  and 2O2-

**One important thing is that the distance matrix are different for type 2,3,4**

Starting initialize a new function to generate different kinds of clusters


In [21]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix



li3clo_210=Structure.from_file("CONTCAR")
li3clo_210.add_oxidation_state_by_guess()
#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})

clusters=[]



for i in range(0,len(li3clo_210)):
    if "Li+" in li3clo_210[i].species:
        nn=local_env_finder.get_nn_info(li3clo_210,i)
        nn_elements=[neighbor["site"].species.elements[0] for neighbor in nn]
        #print(local_env_finder.get_cn_dict(li3clo_210,i))
        
        if local_env_finder.get_cn_dict(li3clo_210,i) not in clusters:
            clusters.append(local_env_finder.get_cn_dict(li3clo_210,i))
print(clusters)

[{'Li+': 8, 'Cl-': 4}, {'Li+': 7, 'Cl-': 4}, {'Cl-': 4, 'Li+': 4}, {'Li+': 6, 'Cl-': 2}]


# A new attempt on rearranging neighbor

In [32]:
import numpy as np

import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools

def build_distance_matrix_from_getnninfo_output(cutoffdnn_output=[],verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix


nasicon=Structure.from_cif("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)

center_Na1=[0,1]
local_env_finder = CutOffDictNN({('Na+','Na+'):4,('Na+','Si4+'):4})


reference_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[0]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"])     

reference_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences)

print(reference_neighbor_sequences)
np.set_printoptions(precision=2,suppress=True)

reference_na_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences[0:6])


possible_arrange_na=[]

for possible_na_sequence in itertools.permutations(reference_neighbor_sequences[0:6]):
    if np.allclose(build_distance_matrix_from_getnninfo_output(possible_na_sequence),reference_na_distance_matrix,rtol=0.01):
        possible_arrange_na.append(possible_na_sequence)
        
print(len(possible_arrange_na))

reference_si_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences[6:12])


possible_arrange_si=[]

for possible_si_sequence in itertools.permutations(reference_neighbor_sequences[6:12]):
    if np.allclose(build_distance_matrix_from_getnninfo_output(possible_si_sequence),reference_si_distance_matrix,rtol=0.01):
        possible_arrange_si.append(possible_si_sequence)

print(len(possible_arrange_si))


total_iter=0

for possible_complete_sequence in itertools.product(possible_arrange_na,possible_arrange_si):
    re_sorted_neighbors_list=[]
    
    for neighbor in possible_complete_sequence:

        re_sorted_neighbors_list.extend(list(neighbor))     

    if np.allclose(build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list),reference_distance_matrix,rtol=0.01):
        print(possible_complete_sequence)
        total_iter+=1
    pass

print(total_iter)
        



[{'site': PeriodicSite: Na+ (1.2831, 2.6519, 12.9389) [0.8897, 0.6103, 0.2500], 'image': (0, 0, 0), 'weight': 3.4778161424304033, 'site_index': 2, 'wyckoff_sequence': 0, 'local_index': 18, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (1.6550, -2.4371, 12.9389) [0.6103, 0.2500, 0.8897], 'image': (0, 0, 0), 'weight': 3.4778161424304033, 'site_index': 3, 'wyckoff_sequence': 1, 'local_index': 19, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (-2.9381, -0.2148, 12.9389) [0.2500, 0.8897, 0.6103], 'image': (0, 0, 0), 'weight': 3.477816142430404, 'site_index': 4, 'wyckoff_sequence': 2, 'local_index': 20, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (-1.6550, 2.4371, 9.2421) [0.3897, 0.7500, 0.1103], 'image': (0, 0, 0), 'weight': 3.4778161424304015, 'site_index': 5, 'wyckoff_sequence': 3, 'local_index': 21, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (2.9381, 0.2148, 9.2421) [0.7500, 0.1103, 0.3897], 'image': (0, 0, 0), 'weight': 3.4778161424304006, 'site_index': 6, 'wyckoff_sequence': 4, 'local_in

# wrap the distance function etc to a class

In [2]:
import numpy as np

import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools


class cluster_matcher():
    
    def __init__(self,neighbor_species=(('Cl-', 4),('Li+', 8)),reference_distance_matrix=np.array([[0,1],[1,0]]),reference_neighbor_sequence=[{}],neighbor_species_respective_reference_distance_matrix_dict={"Cl-":np.array([[0,1],[1,0]]),"Li+":np.array([[0,1],[1,0]])},neighbor_species_respective_reference_neighbor_sequence_dict={"Cl-":[{}],"Li+":[{}]}):
        """cluster matcher, the __init__ method shouln't be used. Use the from_reference_cluster() instead. This is cluster matcher to match the nearest neighbor info output from local_env.cutoffdictNN.get_nn_info. This cluster_matcher Class is initialized by a reference cluster, a distance matrix is built as reference. Then user can call the cluster_matcher.brutal_match function to sort another nn_info so that the sequence of neighbor of "another nn_info" is arranged so that the distance matrix are the same 

        Args:
            neighbor_species (tuple, optional): tuple ( tuple ( str(species), int(number_of_this_specie in neighbors)  )  ). Defaults to (('Cl-', 4),('Li+', 8)).
            reference_distance_matrix (np.array, optional): np.2d array as distance matrix. Defaults to np.array([[0,1],[1,0]]).
            reference_neighbor_sequence (list, optional): list of dictionary in the format of nn_info returning value. Defaults to [{}].
            neighbor_species_respective_reference_distance_matrix_dict (dict, optional): this is a dictionary with key=species and value=distance_matrix(2D numpy array) which record the distance matrix of respective element. . Defaults to {"Cl-":np.array([[0,1],[1,0]]),"Li+":np.array([[0,1],[1,0]])}.
            neighbor_species_respective_reference_neighbor_sequence_dict (dict, optional): dictionary with key=species and value=list of dictionary which is just group the reference neighbor sequence by different elements. Defaults to {"Cl-":[{}],"Li+":[{}]}.
        """
        
        self.neighbor_species=neighbor_species
        self.reference_distance_matrix=reference_distance_matrix
        self.neighbor_species_respective_reference_distance_matrix_dict=neighbor_species_respective_reference_distance_matrix_dict
        self.neighbor_species_respective_reference_neighbor_sequence_dict=neighbor_species_respective_reference_neighbor_sequence_dict
        self.reference_neighbor_sequence=reference_neighbor_sequence
        
        

        
        pass
    
    @classmethod
    def from_reference_neighbor_sequences(self,reference_neighbor_sequences=[{}]):
        cn_dict={}
        neighbor_species_respective_reference_neighbor_sequence_dict={}
        
        for neighbor in reference_neighbor_sequences:
            
            site_element = neighbor["site"].species_string
            
            if site_element not in cn_dict:
                cn_dict[site_element] = 1
            else:
                cn_dict[site_element] += 1
            if site_element not in neighbor_species_respective_reference_neighbor_sequence_dict:
                neighbor_species_respective_reference_neighbor_sequence_dict[site_element]=[neighbor]
            else:
                neighbor_species_respective_reference_neighbor_sequence_dict[site_element].append(neighbor)
        
            
        neighbor_species_respective_reference_distance_matrix_dict={}
        
        for species in neighbor_species_respective_reference_neighbor_sequence_dict:
            neighbor_species_respective_reference_distance_matrix_dict[species]=self.build_distance_matrix_from_getnninfo_output(neighbor_species_respective_reference_neighbor_sequence_dict[species])
        
        neighbor_species=tuple(sorted(cn_dict.items(),key=lambda x:x[0]))
        
        reference_distance_matrix=self.build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences)
        
                
        return cluster_matcher(neighbor_species=neighbor_species,reference_distance_matrix=reference_distance_matrix,reference_neighbor_sequence=reference_neighbor_sequences,neighbor_species_respective_reference_distance_matrix_dict=neighbor_species_respective_reference_distance_matrix_dict,neighbor_species_respective_reference_neighbor_sequence_dict=neighbor_species_respective_reference_neighbor_sequence_dict)
        
    
    @classmethod
    def build_distance_matrix_from_getnninfo_output(self,cutoffdnn_output=[{}]):
        """build a distance matrix from the output of CutOffDictNN.get_nn_info

        nn_info looks like: 
        [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
        
        or say:
        
        nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
        
        Use the site.distance function to build matrix
        

        Args:
            cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

        Returns:
            np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
        """
    
        distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
          

        for sitedictindex1 in range(0,len(cutoffdnn_output)):
            for sitedictindex2 in range(0,sitedictindex1):
                """Reason for jimage=[0,0,0]
                
                site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

                .
                """
                distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
            
            
        
        return distance_matrix

    def brutal_match(self,unsorted_nninfo=[{}],rtol=0.01):
        """brutally sort the inputted unsorted_nninfo. Although brutal but fast enough for now

        Args:
            unsorted_nninfo (list, optional): the unsorted nn_info of an element. The nn_info are compared with the nn_info of class instance. Defaults to [{}].
            rtol (float, optional): rtolerance of np.allclose in order to determine if the distance matrix are the same. Better not too small. Defaults to 0.01.

        Raises:
            ValueError: this will perform a check if the inputted unsorted nn_info has the same neighbor species type and amount
            ValueError: if the unsorted_nninfo cannot be sort with reference of the distance matrix of this cluster_matcher instance. Probably due to too small rtol or it's just not the same clusters

        Returns:
            list of dictionary: in the format of cutoffdictNN.get_nn_info
        """
        
        unsorted_cluster=cluster_matcher.from_reference_neighbor_sequences(unsorted_nninfo)
        
        if self.neighbor_species!=unsorted_cluster.neighbor_species:
            raise ValueError("input cluster has different environment")
        
        sorted_neighbor_sequence_dict={}

        for specie in unsorted_cluster.neighbor_species_respective_reference_neighbor_sequence_dict:
            
            sorted_neighbor_sequence_dict[specie]=[]
            
            for possible_local_sequence in itertools.permutations(unsorted_cluster.neighbor_species_respective_reference_neighbor_sequence_dict[specie]):
                
                if np.allclose(self.build_distance_matrix_from_getnninfo_output(possible_local_sequence),self.neighbor_species_respective_reference_distance_matrix_dict[specie],rtol=rtol):
                    
                    sorted_neighbor_sequence_dict[specie].append(list(possible_local_sequence))
        
        #logging.warning(str(sorted_neighbor_sequence_dict))
        
        
        sorted_neighbor_sequence_list=[]
        
        for specie in sorted_neighbor_sequence_dict:
            sorted_neighbor_sequence_list.append(sorted_neighbor_sequence_dict[specie])
        
        for possible_complete_sequence in itertools.product(*sorted_neighbor_sequence_list):
            
            #logging.warning(str(possible_complete_sequence))
            
            re_sorted_neighbors_list=[]
            
            for neighbor in possible_complete_sequence:

                re_sorted_neighbors_list.extend(list(neighbor))               
        
            if np.allclose(self.build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list),self.reference_distance_matrix,rtol=rtol):
                return re_sorted_neighbors_list
  
        raise ValueError("sequence not founded!")                
        

nasicon=Structure.from_cif("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)

center_Na1=[0,1]
local_env_finder = CutOffDictNN({('Na+','Na+'):4,('Na+','Si4+'):4})


reference_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[0]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"])     

np.set_printoptions(precision=2)
a=cluster_matcher.from_reference_neighbor_sequences(reference_neighbor_sequences=reference_neighbor_sequences)


print(a.neighbor_species)
print(a.reference_distance_matrix)
print(a.neighbor_species_respective_reference_distance_matrix_dict)
print(a.neighbor_species_respective_reference_neighbor_sequence_dict)


wrong_sequence_neighbor=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[1]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"]) 


a.brutal_match(wrong_sequence_neighbor,rtol=0.01)


(('Na+', 6), ('Si4+', 6))
[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [5.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [5.1  5.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [4.73 6.96 4.73 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [4.73 4.73 6.96 5.1  0.   0.   0.   0.   0.   0.   0.   0.  ]
 [6.96 4.73 4.73 5.1  5.1  0.   0.   0.   0.   0.   0.   0.  ]
 [3.16 6.2  3.06 3.71 6.54 6.49 0.   0.   0.   0.   0.   0.  ]
 [3.06 3.16 6.2  6.49 3.71 6.54 5.63 0.   0.   0.   0.   0.  ]
 [6.2  3.06 3.16 6.54 6.49 3.71 5.63 5.63 0.   0.   0.   0.  ]
 [6.54 6.49 3.71 3.16 6.2  3.06 4.92 7.48 4.92 0.   0.   0.  ]
 [3.71 6.54 6.49 3.06 3.16 6.2  4.92 4.92 7.48 5.63 0.   0.  ]
 [6.49 3.71 6.54 6.2  3.06 3.16 7.48 4.92 4.92 5.63 5.63 0.  ]]
{'Na+': array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [5.1 , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [5.1 , 5.1 , 0.  , 0.  , 0.  , 0.  ],
       [4.73, 6.96, 4.73, 0.  , 0.  , 0.  ],
       [4.73, 4.73, 6.96, 5.1

[{'site': PeriodicSite: Na+ (1.2831, -2.6519, -1.8484) [-0.1103, -0.3897, 0.2500],
  'image': (-1, -1, 0),
  'weight': 3.477816142430401,
  'site_index': 2,
  'wyckoff_sequence': 0,
  'local_index': 18,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (1.6550, 2.4371, -1.8484) [0.2500, -0.1103, -0.3897],
  'image': (0, -1, -1),
  'weight': 3.4778161424303997,
  'site_index': 4,
  'wyckoff_sequence': 2,
  'local_index': 20,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (-2.9381, 0.2148, -1.8484) [-0.3897, 0.2500, -0.1103],
  'image': (-1, 0, -1),
  'weight': 3.4778161424304,
  'site_index': 3,
  'wyckoff_sequence': 1,
  'local_index': 19,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (-1.6550, -2.4371, 1.8484) [-0.2500, 0.1103, 0.3897],
  'image': (-1, 0, 0),
  'weight': 3.477816142430402,
  'site_index': 6,
  'wyckoff_sequence': 4,
  'local_index': 22,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (2.9381, -0.2148, 1.8484) [0.3897, -0.2500, 0.1103],
  'image': (0, -1, 0),
  'weight': 

# generate events version 3

In [8]:
import numpy as np
import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools
from kmcpy.io import convert
from kmcpy.event import Event
from kmcpy.event_generator import generate_event_kernal



class cluster_matcher():
    
    def __init__(self,neighbor_species=(('Cl-', 4),('Li+', 8)),reference_distance_matrix=np.array([[0,1],[1,0]]),reference_neighbor_sequence=[{}],neighbor_species_respective_reference_distance_matrix_dict={"Cl-":np.array([[0,1],[1,0]]),"Li+":np.array([[0,1],[1,0]])},neighbor_species_respective_reference_neighbor_sequence_dict={"Cl-":[{}],"Li+":[{}]}):
        """cluster matcher, the __init__ method shouln't be used. Use the from_reference_cluster() instead. This is cluster matcher to match the nearest neighbor info output from local_env.cutoffdictNN.get_nn_info. This cluster_matcher Class is initialized by a reference cluster, a distance matrix is built as reference. Then user can call the cluster_matcher.brutal_match function to sort another nn_info so that the sequence of neighbor of "another nn_info" is arranged so that the distance matrix are the same 

        Args:
            neighbor_species (tuple, optional): tuple ( tuple ( str(species), int(number_of_this_specie in neighbors)  )  ). Defaults to (('Cl-', 4),('Li+', 8)).
            reference_distance_matrix (np.array, optional): np.2d array as distance matrix. Defaults to np.array([[0,1],[1,0]]).
            reference_neighbor_sequence (list, optional): list of dictionary in the format of nn_info returning value. Defaults to [{}].
            neighbor_species_respective_reference_distance_matrix_dict (dict, optional): this is a dictionary with key=species and value=distance_matrix(2D numpy array) which record the distance matrix of respective element. . Defaults to {"Cl-":np.array([[0,1],[1,0]]),"Li+":np.array([[0,1],[1,0]])}.
            neighbor_species_respective_reference_neighbor_sequence_dict (dict, optional): dictionary with key=species and value=list of dictionary which is just group the reference neighbor sequence by different elements. Defaults to {"Cl-":[{}],"Li+":[{}]}.
        """
        
        self.neighbor_species=neighbor_species
        self.reference_distance_matrix=reference_distance_matrix
        self.neighbor_species_respective_reference_distance_matrix_dict=neighbor_species_respective_reference_distance_matrix_dict
        self.neighbor_species_respective_reference_neighbor_sequence_dict=neighbor_species_respective_reference_neighbor_sequence_dict
        self.reference_neighbor_sequence=reference_neighbor_sequence
        
    
        
        pass
    
    @classmethod
    def from_reference_neighbor_sequences(self,reference_neighbor_sequences=[{}]):
        cn_dict={}
        neighbor_species_respective_reference_neighbor_sequence_dict={}
        
        for neighbor in reference_neighbor_sequences:
            
            site_element = neighbor["site"].species_string
            
            if site_element not in cn_dict:
                cn_dict[site_element] = 1
            else:
                cn_dict[site_element] += 1
            if site_element not in neighbor_species_respective_reference_neighbor_sequence_dict:
                neighbor_species_respective_reference_neighbor_sequence_dict[site_element]=[neighbor]
            else:
                neighbor_species_respective_reference_neighbor_sequence_dict[site_element].append(neighbor)
        
            
        neighbor_species_respective_reference_distance_matrix_dict={}
        
        for species in neighbor_species_respective_reference_neighbor_sequence_dict:
            neighbor_species_respective_reference_distance_matrix_dict[species]=self.build_distance_matrix_from_getnninfo_output(neighbor_species_respective_reference_neighbor_sequence_dict[species])
        
        neighbor_species=tuple(sorted(cn_dict.items(),key=lambda x:x[0]))
        
        reference_distance_matrix=self.build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences)
        
                
        return cluster_matcher(neighbor_species=neighbor_species,reference_distance_matrix=reference_distance_matrix,reference_neighbor_sequence=reference_neighbor_sequences,neighbor_species_respective_reference_distance_matrix_dict=neighbor_species_respective_reference_distance_matrix_dict,neighbor_species_respective_reference_neighbor_sequence_dict=neighbor_species_respective_reference_neighbor_sequence_dict)
        
    
    @classmethod
    def build_distance_matrix_from_getnninfo_output(self,cutoffdnn_output=[{}]):
        """build a distance matrix from the output of CutOffDictNN.get_nn_info

        nn_info looks like: 
        [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
        
        or say:
        
        nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
        
        Use the site.distance function to build matrix
        

        Args:
            cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

        Returns:
            np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
        """
    
        distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
          

        for sitedictindex1 in range(0,len(cutoffdnn_output)):
            for sitedictindex2 in range(0,sitedictindex1):
                """Reason for jimage=[0,0,0]
                
                site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

                .
                """
                distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
            
            
        
        return distance_matrix

    def brutal_match(self,unsorted_nninfo=[{}],rtol=0.01,atol=0.01,find_nearest_if_fail=True):
        """brutally sort the inputted unsorted_nninfo. Although brutal but fast enough for now

        Args:
            unsorted_nninfo (list, optional): the unsorted nn_info of an element. The nn_info are compared with the nn_info of class instance. Defaults to [{}].
            rtol (float, optional): rtolerance of np.allclose in order to determine if the distance matrix are the same. Better not too small. Defaults to 0.01.

        Raises:
            ValueError: this will perform a check if the inputted unsorted nn_info has the same neighbor species type and amount
            ValueError: if the unsorted_nninfo cannot be sort with reference of the distance matrix of this cluster_matcher instance. Probably due to too small rtol or it's just not the same clusters

        Returns:
            list of dictionary: in the format of cutoffdictNN.get_nn_info
        """
        
        unsorted_cluster=cluster_matcher.from_reference_neighbor_sequences(unsorted_nninfo)
        
        if self.neighbor_species!=unsorted_cluster.neighbor_species:
            raise ValueError("input cluster has different environment")
        
        if np.allclose(unsorted_cluster.reference_distance_matrix,self.reference_distance_matrix,rtol=rtol,atol=atol):
            logging.info("no need to rearrange this cluster. The distance matrix is already the same. The differece matrix is : \n")
            logging.info(str(unsorted_cluster.reference_distance_matrix-self.reference_distance_matrix))
            
            return unsorted_cluster.reference_neighbor_sequence
        
        
        
        sorted_neighbor_sequence_dict={}

        for specie in unsorted_cluster.neighbor_species_respective_reference_neighbor_sequence_dict:
            
            sorted_neighbor_sequence_dict[specie]=[]

            for possible_local_sequence in itertools.permutations(unsorted_cluster.neighbor_species_respective_reference_neighbor_sequence_dict[specie]):
                
                if np.allclose(self.build_distance_matrix_from_getnninfo_output(possible_local_sequence),self.neighbor_species_respective_reference_distance_matrix_dict[specie],rtol=rtol,atol=atol):
                    
                    sorted_neighbor_sequence_dict[specie].append(list(possible_local_sequence))
            
            if len(sorted_neighbor_sequence_dict[specie])==0:
                raise ValueError("no sorted sequence found for "+str(specie)+" please check if the rtol or atol is too small")
            
        #logging.warning(str(sorted_neighbor_sequence_dict))
        
        
        sorted_neighbor_sequence_list=[]
        
        for specie in sorted_neighbor_sequence_dict:
            sorted_neighbor_sequence_list.append(sorted_neighbor_sequence_dict[specie])
        
        

        if find_nearest_if_fail:
            closest_smilarity_score=999999.0
            closest_sequence=[]
            for possible_complete_sequence in itertools.product(*sorted_neighbor_sequence_list):
                
  
                re_sorted_neighbors_list=[]
                
                for neighbor in possible_complete_sequence:

                    re_sorted_neighbors_list.extend(list(neighbor))               
            
                this_smilarity_score=np.sum(np.abs(self.build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list)-self.reference_distance_matrix))
                
                if this_smilarity_score<closest_smilarity_score:
                    closest_smilarity_score=this_smilarity_score
                    closest_sequence=re_sorted_neighbors_list
                    
            logging.warning("the closest cluster identified. Total difference"+str(closest_smilarity_score))
            logging.info("new sorting is found,new distance matrix is ")
            logging.info(str(self.build_distance_matrix_from_getnninfo_output(closest_sequence)))
            logging.info("The differece matrix is : \n")
            logging.info(str(self.build_distance_matrix_from_getnninfo_output(closest_sequence)-self.reference_distance_matrix))                            
            return closest_sequence
                
            
      
        else:
        
            for possible_complete_sequence in itertools.product(*sorted_neighbor_sequence_list):
                
                #logging.warning(str(possible_complete_sequence))
                
                re_sorted_neighbors_list=[]
                
                for neighbor in possible_complete_sequence:

                    re_sorted_neighbors_list.extend(list(neighbor))               
            
                if np.allclose(self.build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list),self.reference_distance_matrix,rtol=rtol,atol=atol):
                    logging.info("new sorting is found,new distance matrix is ")
                    logging.info(str(self.build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list)))
                    logging.warning("The differece matrix is : \n")
                    logging.info(str(self.build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list)-self.reference_distance_matrix))                
                    
                    return re_sorted_neighbors_list
  
            raise ValueError("sequence not founded!")                
   

def extract_clusters_from_primitive_cell(prim_cif_name="210.cif",local_env_cutoff_dict={("Li+","Cl-"):4.0,("Li+","Li+"):3.0},atom_identifier_type="specie",center_atom_identifier="Li+",diffuse_to_atom_identifier="Li+",species_to_be_removed=["O2-","O"],distance_matrix_rtol=0.01,distance_matrix_atol=0.01,find_nearest_if_fail=True,convert_to_primitive_cell=False,create_reference_cluster=True,supercell_shape=[2,1,1],event_fname="events.json",event_kernal_fname='event_kernal.csv',verbosity=0):

    # --------------
    
    
    logging.basicConfig(level=verbosity,handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ])
    logging.warning("Extracting clusters from primitive cell structure. This primitive cell can also be grain boundary")
    
    primitive_cell=Structure.from_cif(prim_cif_name,primitive=convert_to_primitive_cell)
    primitive_cell.add_oxidation_state_by_guess()
    primitive_cell.remove_species(species_to_be_removed)
    
    logging.warning("primitive cell composition after adding oxidation state and removing uninvolved species: ")
    
    logging.info(primitive_cell.composition)
    
    logging.warning("building center atom index list")
    
    
    center_atom_indices=[]    
    if atom_identifier_type=="specie":
        for i in range(0,len(primitive_cell)):
            if center_atom_identifier in primitive_cell[i].species:
                center_atom_indices.append(i)
                
    elif atom_identifier_type=="label":

        for i in range(0,len(primitive_cell)):
            if primitive_cell[i].properties["label"]==center_atom_identifier:
                center_atom_indices.append(i)
                
    elif atom_identifier_type=="list":
        center_atom_indices=center_atom_identifier
    
    else:
        raise ValueError('unrecognized atom_identifier_type. Please select from: ["specie","label","list"] ')
    
    logging.info("please check if these are center atom:")
    for i in center_atom_indices:
        
        logging.info(str(primitive_cell[i]))        
        
        
        
    #--------
    
    local_env_finder = CutOffDictNN(local_env_cutoff_dict)
    
    reference_cluster_dict={}
    
    local_env_info_dict = {}
   
   
    reference_cluster_type=0
    
    logging.info("start finding the neighboring sequence of center atoms")
    logging.info("total number of center atoms:"+str(len(center_atom_indices)))
    neighbor_has_been_found=0
    
    for center_atom_index in center_atom_indices:
        
        unsorted_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(primitive_cell,center_atom_index),key=lambda x:x["weight"]),key = lambda x:x["label"])      
                    
        this_cluster=cluster_matcher.from_reference_neighbor_sequences(unsorted_neighbor_sequences)
        
        
        
        if this_cluster.neighbor_species not in reference_cluster_dict:
            
            if create_reference_cluster:
                reference_cluster_sites=[primitive_cell[center_atom_index]]
                
                for i in unsorted_neighbor_sequences:
                    
                    reference_cluster_sites.append(i["site"])
                    reference_cluster_structure=Structure.from_sites(sites=reference_cluster_sites)
                    
                reference_cluster_structure.to("cif",str(reference_cluster_type)+"th_reference_cluster.cif") 
                reference_cluster_type+=1
                
                logging.info(str(reference_cluster_type)+"th type of reference cluster structure cif file is created. please check")
            
            reference_cluster_dict[this_cluster.neighbor_species]=this_cluster

            local_env_info_dict[primitive_cell[center_atom_index].properties['local_index']]=this_cluster.reference_neighbor_sequence
            
            
            logging.warning("a new type of cluster is recognized with the species "+str(this_cluster.neighbor_species)+" \nthe distance matrix are \n"+str(this_cluster.reference_distance_matrix))

            
        else:
            logging.info("a cluster is created with the species "+str(this_cluster.neighbor_species)+" \nthe distance matrix are \n"+str(this_cluster.reference_distance_matrix))
            
            

            sorted_neighbor_sequence=reference_cluster_dict[this_cluster.neighbor_species].brutal_match(this_cluster.reference_neighbor_sequence,rtol=distance_matrix_rtol,atol=distance_matrix_atol,find_nearest_if_fail=find_nearest_if_fail)
            
            local_env_info_dict[primitive_cell[center_atom_index].properties['local_index']]=sorted_neighbor_sequence
            
        neighbor_has_been_found+=1
        logging.warning(str(neighbor_has_been_found)+" out of "+str(len(center_atom_indices))+" neighboring sequence has been found")
     

    #return local_env_info_dict          

  
    supercell=primitive_cell.make_kmc_supercell(supercell_shape)
    logging.warning("supercell is created")
    logging.info(str(supercell))
    
    supercell_center_atom_indices=[]
    if atom_identifier_type=="specie":
        for i in range(0,len(supercell)):
            if center_atom_identifier in supercell[i].species:
                supercell_center_atom_indices.append(i)
                
    elif atom_identifier_type=="label":

        for i in range(0,len(supercell)):
            if supercell[i].properties["label"]==center_atom_identifier:
                supercell_center_atom_indices.append(i)
                
    elif atom_identifier_type=="list":
        supercell_center_atom_indices=center_atom_identifier
    
    else:
        raise ValueError('unrecognized atom_identifier_type. Please select from: ["specie","label","list"] ')

            
    events = []
    events_dict = []
    
    def _equivalent_position_in_periodic_supercell(site_belongs_to_supercell=[5,1,7],image_of_site=(0,-1,1),supercell_shape=[5,6,7],additional_input=False,verbose=False):
        """finding the equivalent position in periodic supercell considering the periodic boundary condition
        input:
        site_belongs_to_supercell: site belongs to which supercell

        Returns:
            _type_: _description_
        """
        if verbose:
            print ("equivalent position",site_belongs_to_supercell,image_of_site)
        # 5 1 7 with image 0 -1 1 -> 5 0 8 -> in periodic 567 supercell should change to 561, suppose supercell start with index1
        
        temp=np.array(site_belongs_to_supercell)+np.array(image_of_site)
        # 517+(0-11)=508
        
        
        # 508-1=4-17 mod: 4 5 0 
        #+1 : 561
        temp=np.mod(temp,supercell_shape)
        
        temp=temp.tolist()
        if additional_input is not False:
            temp.append(additional_input)
        return tuple(temp)    
    
    
    indices_dict_from_identifier=supercell.kmc_build_dict3(skip_check=False)#a dictionary. Key is the tuple with same format as class.kmc_info_to_tuple, Value is the global indices   

    
    for supercell_center_atom_index in supercell_center_atom_indices:
        
        # for center atoms of newly generated supercell, find the neighbors

        this_center_atom_belongs_to_supercell=supercell[supercell_center_atom_index].properties["supercell"]
        
        local_index_of_this_center_atom=supercell[supercell_center_atom_index].properties["local_index"]
        
        local_env_info=[]# list of integer / indices of local environment
        
        
        for neighbor_site_in_primitive_cell in local_env_info_dict[local_index_of_this_center_atom]:
            """
            
             local_env_info_dict[local_index_of_this_center_atom]
            
            In primitive cell, the center atom has 1 unique identifier: The "wyckoff sequence inside the primitive cell"
            
            IN supercell, the center atom has an additional unique identifier: belongs to which supercell
            
            However, as long as the "wyckoff sequence inside the primitive cell" is the same, no matter which supercell this center atom belongs to, the sequence of "wyckoff sequence" of its neighbor sites are the same. In primitive cell, center atom with index 1 has neighbor arranged in 1,3,2,4,6,5, then for every center atom with index 1 in supercell, the neighbor is arranged in 1,3,2,4,6,5
            
            Mapping the sequence in primitive cell to supercell!
            
            
            In order to accelerate the speed

            use the dictionary to store the index of atoms
            
            indices_dict_from_identifier is a dictionary by pymatgen_structure.kmc_build_dict()
            
            the key to dictionary is kmc_info_to_tuple
            
            This loop build the local_env_info list, [supercell_neighbor_index1, supercell_neighbor_index2, .....]
            
            """
            
            local_env_info.append(indices_dict_from_identifier[supercell.kmc_info_to_tuple3(local_index=neighbor_site_in_primitive_cell["local_index"],label=neighbor_site_in_primitive_cell["label"],supercell=_equivalent_position_in_periodic_supercell(site_belongs_to_supercell=this_center_atom_belongs_to_supercell,image_of_site=neighbor_site_in_primitive_cell["image"],supercell_shape=supercell_shape))])

        for local_env in local_env_info:
            # generate event
            
            if atom_identifier_type=="specie":
                if diffuse_to_atom_identifier in supercell[local_env].species  :
                    # or for understanding, if any site in local environment, its label== "Na2"
                    # initialize the event
                    this_event = Event()
                    this_event.initialization2(supercell_center_atom_index,local_env,local_env_info)
                    events.append(this_event)
                    events_dict.append(this_event.as_dict())   
                        
            elif atom_identifier_type=="label":

                if supercell[local_env].properties["label"] == diffuse_to_atom_identifier:
                    # or for understanding, if any site in local environment, its label== "Na2"
                    # initialize the event
                    this_event = Event()
                    this_event.initialization2(supercell_center_atom_index,local_env,local_env_info)
                    events.append(this_event)
                    events_dict.append(this_event.as_dict())   
                        
            elif atom_identifier_type=="list":
                raise NotImplementedError("how to do this....") 
                        
            
            else:
                raise ValueError('unrecognized atom_identifier_type. Please select from: ["specie","label","list"] ')
            
            
            
    
    if len(events)==0:
        raise ValueError("There is no events generated. This is probably caused by wrong input parameters. Probably check the diffuse_to_atom_label?")
    
    print('Saving:',event_fname)
    with open(event_fname,'w') as fhandle:
        jsonStr = json.dumps(events_dict,indent=4,default=convert)
        fhandle.write(jsonStr)
    
    events_site_list = []
    for event in events:
        # sublattice indices: local site index for each site
        events_site_list.append(event.sorted_sublattice_indices)
    
    #np.savetxt('./events_site_list.txt',np.array(events_site_list,dtype=int),fmt="%i") # dimension not equal error
    generate_event_kernal(len(supercell),np.array(events_site_list),event_kernal_fname=event_kernal_fname)    
    
np.set_printoptions(precision=2,suppress=True)

    
primitive_cell_cutoff_dict=extract_clusters_from_primitive_cell(prim_cif_name="210.cif",local_env_cutoff_dict={("Li+","Cl-"):4.0,("Li+","Li+"):3.0},atom_identifier_type="specie",center_atom_identifier="Li+",diffuse_to_atom_identifier="Li+",species_to_be_removed=["O2-","O"],distance_matrix_rtol=0.25,distance_matrix_atol=0.5,find_nearest_if_fail=True,convert_to_primitive_cell=False,create_reference_cluster=True,verbosity=0)

def convert2(o):
    if isinstance(o, np.int64): return int(o)
    elif isinstance(o, np.int32): return int(o)  
    try:
        return o.as_dict()
    except:
        return o


with open("primitive_cell_neighbor.json","w") as f:
    jsonstr=json.dumps(primitive_cell_cutoff_dict,indent=4,default=convert2)
    f.write(jsonstr) 
    

INFO:root:Li+120 Cl-40
INFO:root:please check if these are center atom:
INFO:root:[1.92 3.51 6.43] Li+
INFO:root:[ 1.93  3.5  15.15] Li+
INFO:root:[ 1.93  3.53 23.85] Li+
INFO:root:[ 1.93  3.72 32.69] Li+
INFO:root:[1.93 0.05 4.77] Li+
INFO:root:[ 1.93  0.01 13.4 ] Li+
INFO:root:[ 1.93  0.04 22.11] Li+
INFO:root:[ 1.93  0.01 30.75] Li+
INFO:root:[1.93 5.04 2.83] Li+
INFO:root:[ 1.93  5.24 11.67] Li+
INFO:root:[ 1.93  5.26 20.37] Li+
INFO:root:[ 1.93  5.26 29.08] Li+
INFO:root:[1.93 1.76 9.93] Li+
INFO:root:[ 1.93  1.77 18.63] Li+
INFO:root:[ 1.93  1.82 27.34] Li+
INFO:root:[1.93 6.95 8.18] Li+
INFO:root:[ 1.93  6.99 16.89] Li+
INFO:root:[ 1.93  7.01 25.58] Li+
INFO:root:[1.92 6.12 5.5 ] Li+
INFO:root:[ 1.93  6.11 14.28] Li+
INFO:root:[ 1.93  6.14 22.97] Li+
INFO:root:[ 1.93  6.14 31.72] Li+
INFO:root:[1.92 2.63 3.79] Li+
INFO:root:[ 1.93  2.62 12.54] Li+
INFO:root:[ 1.93  2.65 21.24] Li+
INFO:root:[ 1.93  2.64 30.01] Li+
INFO:root:[ 1.93  7.85 10.79] Li+
INFO:root:[ 1.93  7.87 19.5 ] L

Saving: events.json
Generating event kernal ...


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1311)
           2	LOAD_FAST(arg=0, lineno=1311)
           4	STORE_FAST(arg=2, lineno=1311)
           6	LOAD_GLOBAL(arg=0, lineno=1312)
           8	LOAD_METHOD(arg=1, lineno=1312)
          10	LOAD_FAST(arg=2, lineno=1312)
          12	CALL_METHOD(arg=1, lineno=1312)
          14	STORE_FAST(arg=3, lineno=1312)
          16	LOAD_GLOBAL(arg=2, lineno=1313)
          18	CALL_FUNCTION(arg=0, lineno=1313)
          20	STORE_FAST(arg=4, lineno=1313)
          22	LOAD_FAST(arg=3, lineno=1314)
          24	GET_ITER(arg=None, lineno=1314)
>         26	FOR_ITER(arg=80, lineno=1314)
          28	STORE_FAST(arg=5, lineno=1314)
          30	LOAD_GLOBAL(arg=2, lineno=1316)
          32	CALL_FUNCTION(arg=0, lineno=1316)
          34	STORE_FAST(arg=6, lineno=1316)
          36	LOAD_CONST(arg=1, lineno=1317)
          38	STORE_FAST(arg=7, lineno=1317)
          40	LOAD_CONST(arg=2, lineno=1318)
          42	STORE_FAST(arg=8, 

test on new method of rearranging neighbor

In [26]:
import json

def convert(o):
    if isinstance(o, np.int64): return int(o)
    elif isinstance(o, np.int32): return int(o)  
    return o
def convert2(o):
    if isinstance(o, np.int64): return int(o)
    elif isinstance(o, np.int32): return int(o)  
    try:
        return o.as_dict()
    except:
        return o


with open("primitive_cell_neighbor.json","w") as f:
    jsonstr=json.dumps(primitive_cell_cutoff_dict,indent=4,default=convert2)
    f.write(jsonstr)

raise NotImplementedError


primitive_cell_cutoff_dict_jsonable={}

for center_atom_index in primitive_cell_cutoff_dict:
    primitive_cell_cutoff_dict_jsonable[center_atom_index]=[]
    for neighbor in primitive_cell_cutoff_dict[center_atom_index]:
        this_neighbor={}
        for information in neighbor:
            if information=="site":
                this_neighbor[information]=neighbor[information].as_dict()
            
            else:
                
                this_neighbor[information]=neighbor[information]
        primitive_cell_cutoff_dict_jsonable[center_atom_index].append(this_neighbor)




with open("primitive_cell_neighbor.json","w+") as f:
    jsonstr=json.dumps(primitive_cell_cutoff_dict_jsonable,indent=4,default=convert)
    f.write(jsonstr)


In [6]:
a=[[0,1],[0]]
np.array(a)


/var/folders/c5/kgz9vbc90pzfjldxjw_769w00000gn/T/ipykernel_28405/1250423874.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(a)


array([list([0, 1]), list([0])], dtype=object)